In [ ]:
pip install sqlite3

In [ ]:
import requests
import lxml.html

class Games:
    def __init__ (self,titles,prices,totalTags,totalPlatforms):
        self.titles = titles
        self.prices = prices
        self.totalTags = totalTags
        self.totalPlatforms = totalPlatforms

html = requests.get ("https://store.steampowered.com/explore/new/")

doc = lxml.html.fromstring(html.content)

newReleases = doc.xpath('//div[@id="tab_newreleases_content"]')[0]

#print(newReleases)

titles = newReleases.xpath('.//div[@class="tab_item_name"]/text()')

#print(titles)

prices = newReleases.xpath('.//div[@class="discount_final_price"]/text()')

#print(prices)

tags = newReleases.xpath('.//div[@class="tab_item_top_tags"]')

totalTags = []

for tag in tags:
    totalTags.append(tag.text_content())

#print(totalTags)

totalTags = [tag.split(', ') for tag in totalTags]

#print(totalTags)

platformsdiv = newReleases.xpath('.//div[@class="tab_item_details"]')

totalPlatforms = []

for game in platformsdiv:
    namePlatform = game.xpath('.//span[contains(@class, "platform_img")]')
    platforms=[t.get('class').split(' ')[-1] for t in namePlatform]
    if 'had_separator' in platforms:
        platforms.remove('had_separator')
    totalPlatforms.append(platforms)

#print(totalPlatforms)

output = []

for info in zip(titles,prices,totalTags,totalPlatforms):
    response = {}
    response['title'] = info[0]
    response['price'] = info[1]
    response['tags'] = info[2]
    response['platforms'] = info[3]
    output.append(response)

#print(output)

for juego  in zip(titles,prices,totalTags,totalPlatforms):
    juegos = Games(juego[0],juego[1],juego[2],juego[3])
    print("Title: " + juegos.titles)
    print("Price: " + juegos.prices)
    print("Tags: " + str(juegos.totalTags))
    print("Platforms: " + str(juegos.totalPlatforms))

In [ ]:
from bottle import Bottle,run,template,request,redirect

app=Bottle()

@app.route('/')
@app.route('/games')
def game():
    class Games:
        def __init__ (self,titles,prices,totalTags,totalPlatforms):
            self.titles = titles
            self.prices = prices
            self.totalTags = totalTags
            self.totalPlatforms = totalPlatforms

    import requests
    import lxml.html

    html = requests.get ('https://store.steampowered.com/explore/new/')

    #print(html.content)

    doc = lxml.html.fromstring(html.content)

    newReleases = doc.xpath('//div[@id="tab_newreleases_content"]')[0]

    #print(newReleases)

    titles=newReleases.xpath('.//div[@class="tab_item_name"]/text()')

    #print(tittles)

    prices = newReleases.xpath('.//div[@class="discount_final_price"]/text()')

    #print(prices)

    tags = newReleases.xpath('.//div[@class="tab_item_top_tags"]')

    totalTags=[]

    for tag in tags:
        totalTags.append(tag.text_content())

    totalTags = [tag.split(', ') for tag in totalTags]

    #print(totalTags)

    platformsDiv = newReleases.xpath('.//div[@class="tab_item_details"]')
    
    totalPlatforms=[]

    for game in platformsDiv:
        namePlatform = game.xpath('.//span[contains(@class, "platform_img")]')
        platforms=[t.get('class').split(' ')[-1] for t in namePlatform]
        if 'had_separator' in platforms:
            platforms.remove('had separator')
        totalPlatforms.append(platforms)

    
    #print(totalPlatforms)
    gameslist = []

    for info in zip(titles,prices,totalTags,totalPlatforms):
        juegos = Games(info[0],info[1],info[2],info[3])
        gameslist.append(vars(juegos))
    
    output=template('new_releases',rows=gameslist)
    return output
run(app,host='0.0.0.0',port=5000,debug=True)

In [ ]:
import sqlite3
com = sqlite3.connect('games.db')
com.execute("CREATE TABLE games_list (titles varchar(100) NOT NULL,price varchar(100) NOT NULL,tags varchar(100) NOT NULL,platforms varchar(100) NOT NULL)")
com.commit()

In [ ]:
import requests
import lxml.html
import sqlite3
import datetime

class Games:
    def __init__ (self,titles,prices,totalTags,totalPlatforms):
        self.titles = titles
        self.prices = prices
        self.totalTags = totalTags
        self.totalPlatforms = totalPlatforms
def scrapTitles():

    html = requests.get ('https://store.steampowered.com/explore/new/')

    doc = lxml.html.fromstring(html.content)

    newReleases = doc.xpath('//div[@id="tab_newreleases_content"]')[0]

    titles=newReleases.xpath('.//div[@class="tab_item_name"]/text()')

    prices = newReleases.xpath('.//div[@class="discount_final_price"]/text()')

    tags = newReleases.xpath('.//div[@class="tab_item_top_tags"]')

    totalTags=[]

    for tag in tags:
        totalTags.append(tag.text_content())

    totalTags = [tag.split(', ') for tag in totalTags]
    platformsDiv = newReleases.xpath('.//div[@class="tab_item_details"]')

    totalPlatforms=[]

    for game in platformsDiv:
        namePlatform = game.xpath('.//span[contains(@class, "platform_img")]')
        platforms=[t.get('class').split(' ')[-1] for t in namePlatform]
        if 'had_separator' in platforms:
            platforms.remove('had separator')
        totalPlatforms.append(platforms)

    titles_list = []

    for info in zip(titles,prices,totalTags,totalPlatforms):
        juegos = Games(info[0],info[1],info[2],info[3])
        titles_list.append(juegos)

    return titles_list
def update_game_list(game_list):
    
    if len(game_list) > 0:
        for game in game_list:
        
            connection = sqlite3.connect('games.db')
            cursor = connection.cursor()
            tags = ','.join(game.totalTags)
            platforms = ','.join(game.totalPlatforms)
            date = datetime.date.today().strftime("%d/%m/%Y")
            cursor.execute("INSERT INTO games_list (titles,price,tags,platforms,date) VALUES (?,?,?,?,?)",(game.titles,game.prices,tags,platforms,date))
        
            connection.commit()
            cursor.close()
        
if __name__=='__main__':
    game_list = scrapTitles()
    
    update_game_list(game_list)

In [ ]:
from bottle import Bottle,run,template,request,redirect
import sqlite3

def game():
    class Games:
        def __init__ (self,titles,prices,totalTags,totalPlatforms):
            self.titles = titles
            self.prices = prices
            self.totalTags = totalTags
            self.totalPlatforms = totalPlatforms
            
app=Bottle()

@app.route('/games', method ='GET')
def get_list_games():
    connection = sqlite3.connect('games.db')
    cursor = connection.cursor()
    cursor.execute("SELECT DISTINCT titles FROM games_list")
    titles = cursor.fetchall()
    value=[]
    for title in titles:
        value.append(title[0])
    list_titles={}
    list_titles["titles"]=value
    return str(list_titles)

@app.route('/games/date/:date', method ='GET')
def get_list_games(date):
    connection = sqlite3.connect('games.db')
    cursor = connection.cursor()
    cursor.execute(f"SELECT titles, price FROM games_list WHERE date = '{date}'")
    titles = cursor.fetchall()
    value=[]
    for title in titles:
        title_price={title[0]:title[1]}
        value.append(title_price)
    list_data={}
    list_data["titles"]=value
    return str(list_data)

@app.route('/games/history', method ='GET')
def get_game_history():
    data = request.json
    connection = sqlite3.connect('games.db')
    cursor = connection.cursor()
    title_game = data["title"]
    cursor.execute(f"SELECT price, date FROM games_list WHERE titles = '{title_game}'")
    result = cursor.fetchall()
    list_prices=[]
                   
    for title in result:
        gameobj = {title[0]:title[1]}
        list_prices.append(gameobj)
    
    return{title_game: list_prices}
                   
run(app,host='0.0.0.0',port=5000,debug=True)

In [ ]:
from bottle import Bottle,run,template,request,redirect
import sqlite3

def game():
    class Games:
        def __init__ (self,titles,prices,totalTags,totalPlatforms):
            self.titles = titles
            self.prices = prices
            self.totalTags = totalTags
            self.totalPlatforms = totalPlatforms
            
app=Bottle()

@app.route('/games/date', method ='GET')
def get_list_games():
    data = request.json
    connection = sqlite3.connect('games.db')
    cursor = connection.cursor()
    date = data["date"]
    cursor.execute(f"SELECT titles, price FROM games_list WHERE date = '{date}'")
    titles = cursor.fetchall()
    value=[]
    
    for title in titles:
        gameobj = {title[0]:title[1]}
        value.append(gameobj)

    return {date: value}

run(app,host='0.0.0.0',port=5000,debug=True)

In [ ]:
from bottle import Bottle,run,template,request,redirect
import sqlite3

def game():
    class Games:
        def __init__ (self,titles,prices,totalTags,totalPlatforms):
            self.titles = titles
            self.prices = prices
            self.totalTags = totalTags
            self.totalPlatforms = totalPlatforms
            
app=Bottle()
@app.route('/games/update/price',method='POST')
def update_game_price():
    data=request.json
    connection=sqlite3.connect('games.db')
    cursor=connection.cursor()
    title_game=data["title"]
    price=data["price"]
    title_date=data["date"]
    cursor.execute(f"UPDATE games_list SET price='{price}' WHERE titles='{title_game}' AND date='{title_date}'")
    connection.commit()
    cursor.execute(f"SELECT price,date FROM games_list WHERE titles='{title_game}' AND date='{title_date}'")
    titles=cursor.fetchall()
    list_games=[]
    for title in titles:
        gameobj = {"Price":title[0],"Date":title[1]}
        list_games.append(gameobj)
    cursor.close()
    
    return {title_game:list_games}
run(app,host='0.0.0.0',port=5000,debug=True)